__There are 3 criteria to passing the Bechdel Test:__

1. The film has to have at least two [named] women in it,
2. who talk to each other,
3. about something besides a man

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read data 
# can use ExcelFile too
df=pd.read_excel('PD Bechdel Test.xlsx', sheet_name='Webscraping')

In [178]:
code_map=pd.read_excel('PD Bechdel Test.xlsx', sheet_name='html')

In [179]:
code_map.head()

,Char,Numeric,Named,Description
0,NaN,code,code,NaN
1,NaN,&#32;,NaN,space
2,!,&#33;,NaN,exclamation mark
3,"""",&#34;,&quot;,double quote
4,#,&#35;,NaN,number


In [4]:
df.shape

(9417, 2)

In [72]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None
df.head()

,DownloadData,Year,Pass/Fail,Categorisation
0,"<a href=""http://us.imdb.com/title/tt3155794/"">...",1874,fail,Fewer than two women in this movie
1,"<a href=""http://us.imdb.com/title/tt14495706/""...",1877,fail,Fewer than two women in this movie
2,"<a href=""http://us.imdb.com/title/tt12592084/""...",1878,fail,Fewer than two women in this movie
3,"<a href=""http://us.imdb.com/title/tt2221420/"">...",1878,fail,Fewer than two women in this movie
4,"<a href=""http://us.imdb.com/title/tt7816420/"">...",1881,fail,Fewer than two women in this movie


In [16]:
df.dtypes

DownloadData    object
Year             int64
dtype: object

In [274]:
# remove strings after 'return' for each row
Str_parse=df['DownloadData'].str.split('; return',expand=True).iloc[:,0]

In [275]:
# split strings using '"' and find useful columns
Str_parse_filter=Str_parse.str.split('"',expand=True).iloc[:,[3,7,11]]

In [276]:
Str_parse_filter.head()

,3,7,11
0,/static/nopass.png,[Fewer than two women in this movie],/view/9602/passage_de_venus/
1,/static/nopass.png,[Fewer than two women in this movie],/view/9804/la_rosace_magique/
2,/static/nopass.png,[Fewer than two women in this movie],/view/9806/le_singe_musicien/
3,/static/nopass.png,[Fewer than two women in this movie],/view/9603/sallie_gardner_at_a_gallop/
4,/static/nopass.png,[Fewer than two women in this movie],/view/9816/athlete_swinging_a_pick/


In [284]:
# add 'Pass/Fail' column
df['Pass/Fail']=Str_parse_filter[3].str.split('.',n=1, expand=True)[0].str.rsplit('/',n=1, expand=True)[1].values

In [285]:
df['Pass/Fail'].replace('nopass','fail', inplace=True)
#df.loc[df['Pass/Fail']=='nopass','Pass/Fail']='fail'

In [286]:
# find 5 Categorisations
Str_parse_filter[7].unique()

array(['[Fewer than two women in this movie]',
       "[There are two or more women in this movie, but they don't talk to each other]",
       '[There are two or more women in this movie and they talk to each other about something other than a man]',
       '[There are two or more women in this movie, but they only talk to each other about a man]',
       '[There are two or more women in this movie and they talk to each other about something other than a man, although dubious]'],
      dtype=object)

In [287]:
# add Categorisation column
#Method 1
Str_parse_filter[7].str.extract(r'\[(.*)\]').head()

,0
0,Fewer than two women in this movie
1,Fewer than two women in this movie
2,Fewer than two women in this movie
3,Fewer than two women in this movie
4,Fewer than two women in this movie


In [288]:
# add Categorisation column
#Method 2

df['Categorisation']=Str_parse_filter[7].apply(lambda st: st[st.find('[')+1:st.find(']')])

In [290]:
# add Movie Title
#df['Movie Title']=Str_parse_filter[11].str.split('/', expand=True)[3].replace(regex=r'&#39;',value="'")
df['Movie Title']=Str_parse_filter[11].str.split('/', expand=True)[3].replace(regex=r'&amp;',value="&")

In [291]:
# extract the html codes from the Movie titles
# These will always start with a '&' and end with a ';' and maximum length is 6
codes_find=df['Movie Title'].str.extract(r'(&.{,6};)')[0].dropna().unique()

In [292]:
codes_find

array(['&#39;', '&ouml;', '&auml;', '&egrave;', '&ocirc;', '&acute;',
       '&frac12;', '&ograve;', '&#146;', '&eacute;', '&agrave;',
       '&ucirc;', '&iacute;', '&acirc;', '&#378;', '&oslash;', '&aacute;',
       '&amp;', '&euml;', '&quot;', '&#333;', '&aring;', '&uuml;',
       '&ntilde;', '&#305;', '&ecirc;', '&atilde;', '&oacute;', '&#150;'],
      dtype=object)

In [293]:
code_map.loc[(code_map['Numeric']==codes_find[0])|(code_map['Named']==codes_find[0]), 'Char'].unique()[0]

"'"

In [294]:
# use while and for loop to replace the html codes with their correct characters
while len(codes_find)>0:
    for i in range(len(codes_find)):
        C_v=code_map.loc[(code_map['Numeric']==codes_find[i])|(code_map['Named']==codes_find[i]), 'Char'].unique()[0]
        df['Movie Title']=df['Movie Title'].replace(regex=codes_find[i],value=C_v)
    codes_find=df['Movie Title'].str.extract(r'(&.{,6};)')[0].dropna().unique()

In [113]:
# inefficient method:
# one by one replace the html codes with their correct characters
# df['Movie Title']=df['Movie Title'].replace(regex=r'&eacute;',value="\u00e9")
# df['Movie Title']=df['Movie Title'].replace(regex=r'&#378;',value="\u017a")
# df['Movie Title']=df['Movie Title'].replace(regex=r'&amp;',value="&")
# df['Movie Title']=df['Movie Title'].replace(regex=r'&#333;',value="\u014d")
# df['Movie Title']=df['Movie Title'].replace(regex=r'&#305;',value="\u0131")
# df['Movie Title']=df['Movie Title'].replace(regex=r'&#150;',value="-")
# df['Movie Title']=df['Movie Title'].replace(regex=r'&egrave;',value=" ")

In [306]:
# change '_' to space and title
df['Movie Title']=df['Movie Title'].replace(regex=r'_',value=' ').str.title()

In [307]:
df_update=df.drop(columns='DownloadData')

In [308]:
# double check if all the html codes have been replaced 
df_update['Movie Title'][df_update['Movie Title'].str.contains('&')]

613                                      Fun & Fancy Free
1027                                     Advise & Consent
1122                                       King & Country
1411                                 Mccabe & Mrs. Miller
1431                  Willy Wonka & The Chocolate Factory
1514                          Pat Garrett & Billy The Kid
2280                         Planes, Trains & Automobiles
2299                                         Withnail & I
2355                                     Oliver & Company
2390                     Bill & Ted'S Excellent Adventure
2395            The Cook, The Thief, His Wife & Her Lover
2449                                         Tango & Cash
2452                                       Turner & Hooch
2527                  Rosencrantz & Guildenstern Are Dead
2546                           Bill & Ted'S Bogus Journey
2622                                      Thelma & Louise
2717                                         Benny & Joon
2750          

In [309]:
# add rank
df_update.loc[df_update['Categorisation']==df_update['Categorisation'].unique()[2],'Rank']=1
df_update.loc[df_update['Categorisation']==df_update['Categorisation'].unique()[4],'Rank']=2
df_update.loc[df_update['Categorisation']==df_update['Categorisation'].unique()[3],'Rank']=3
df_update.loc[df_update['Categorisation']==df_update['Categorisation'].unique()[1],'Rank']=4
df_update.loc[df_update['Categorisation']==df_update['Categorisation'].unique()[0],'Rank']=5

In [310]:
df_update['Rank']=df_update['Rank'].astype(int)

In [311]:
df_update.shape

(9417, 5)

In [312]:
#sort values based on 'Movie Title','Rank' so that the duplicated films with smaller values of rank can be removed
df_update_sort=df_update.sort_values(by=['Movie Title','Rank'], ascending=False)

In [313]:
# remove duplicated films with smaller values of rank
df_update_sort.drop_duplicates(subset=['Movie Title','Year'], inplace=True)

In [314]:
df_update_sort.shape

(9408, 5)

In [316]:
df_update_sort.head(100)

,Year,Pass/Fail,Categorisation,Movie Title,Rank
6201,2011,fail,Fewer than two women in this movie,Á Annan Veg,5
6926,2012,pass,There are two or more women in this movie and ...,[Rec]3 Genesis,1
5844,2009,fail,"There are two or more women in this movie, but...",[Rec] 2,4
5225,2007,pass,There are two or more women in this movie and ...,[Rec],1
2062,1984,fail,"There are two or more women in this movie, but...",Zwei Nasen Tanken Super,4
4957,2006,pass,There are two or more women in this movie and ...,Zwartboek,1
7340,2013,fail,"There are two or more women in this movie, but...",Zum Geburtstag,3
1146,1964,fail,Fewer than two women in this movie,Zulu,5
1605,1975,fail,"There are two or more women in this movie, but...",Zorro,4
7339,2013,fail,Fewer than two women in this movie,"Zoran, Il Mio Nipote Scemo",5


In [317]:
# output excel
df_update_sort.to_excel('Bechdel Test Output.xlsx', index=False)